In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(81)

### Softmax Loss Function Implement(Low Level) ###

In [2]:
z = torch.FloatTensor([1, 2, 3])

In [3]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)
print(hypothesis.sum())

tensor([0.0900, 0.2447, 0.6652])
tensor(1.)


In [4]:
z = torch.rand(3, 5, requires_grad=True)
print(z)

tensor([[0.9873, 0.7733, 0.2573, 0.0711, 0.8752],
        [0.2900, 0.9377, 0.4219, 0.3910, 0.4411],
        [0.2289, 0.3262, 0.5418, 0.4201, 0.3346]], requires_grad=True)


In [5]:
hypothesis = F.softmax(z, dim=1)
print(hypothesis)
print(hypothesis.sum())

tensor([[0.2791, 0.2253, 0.1345, 0.1116, 0.2495],
        [0.1582, 0.3023, 0.1805, 0.1750, 0.1840],
        [0.1727, 0.1903, 0.2361, 0.2090, 0.1919]], grad_fn=<SoftmaxBackward0>)
tensor(3.0000, grad_fn=<SumBackward0>)


In [6]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([1, 1, 3])


In [7]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.]])

In [8]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4173, grad_fn=<MeanBackward0>)


### Softmax Loss Function Implement(High Level) ###

In [9]:
# torch.log(F.softmax(z, dim=1)) == F.log_softmax(z, dim=1)
torch.log(F.softmax(z, dim=1))

tensor([[-1.2763, -1.4903, -2.0063, -2.1925, -1.3883],
        [-1.8439, -1.1963, -1.7120, -1.7430, -1.6929],
        [-1.7564, -1.6592, -1.4435, -1.5652, -1.6508]], grad_fn=<LogBackward0>)

In [10]:
F.log_softmax(z, dim=1)

tensor([[-1.2763, -1.4903, -2.0063, -2.1925, -1.3883],
        [-1.8439, -1.1963, -1.7120, -1.7430, -1.6929],
        [-1.7564, -1.6592, -1.4435, -1.5652, -1.6508]],
       grad_fn=<LogSoftmaxBackward0>)

In [11]:
# First
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

# Second
cost = (y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()
print(cost)

# Third
cost = F.nll_loss(F.log_softmax(z, dim=1), y)
print(cost)

# Fourth
cost = F.cross_entropy(z, y)
print(cost)

tensor(1.4173, grad_fn=<MeanBackward0>)
tensor(1.4173, grad_fn=<MeanBackward0>)
tensor(1.4173, grad_fn=<NllLossBackward0>)
tensor(1.4173, grad_fn=<NllLossBackward0>)


### EXAMPLE

In [12]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [13]:
# One-Hot-Encoding; y_train에서 클래스는 0, 1, 2 총 3개이므로 결과는 8x3 Tensor가 나와야 함

y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape)

torch.Size([8, 3])


In [14]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros((1, 3), requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [15]:
# Low Level
nb_epochs = 1000
for epoch in range(nb_epochs+1):
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.704200
Epoch  200/1000 Cost: 0.623000
Epoch  300/1000 Cost: 0.565717
Epoch  400/1000 Cost: 0.515291
Epoch  500/1000 Cost: 0.467662
Epoch  600/1000 Cost: 0.421278
Epoch  700/1000 Cost: 0.375402
Epoch  800/1000 Cost: 0.329765
Epoch  900/1000 Cost: 0.285072
Epoch 1000/1000 Cost: 0.248155


In [16]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros((1, 3), requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [17]:
# High Level
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.704199
Epoch  200/1000 Cost: 0.622999
Epoch  300/1000 Cost: 0.565717
Epoch  400/1000 Cost: 0.515291
Epoch  500/1000 Cost: 0.467662
Epoch  600/1000 Cost: 0.421278
Epoch  700/1000 Cost: 0.375401
Epoch  800/1000 Cost: 0.329766
Epoch  900/1000 Cost: 0.285073
Epoch 1000/1000 Cost: 0.248155


In [18]:
# nn.Module

model = nn.Linear(4, 3)
optimizer = optim.SGD(model.parameters(), lr=0.1)
print(optimizer)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)
Epoch    0/1000 Cost: 1.576931
Epoch  100/1000 Cost: 0.680078
Epoch  200/1000 Cost: 0.584699
Epoch  300/1000 Cost: 0.526792
Epoch  400/1000 Cost: 0.480794
Epoch  500/1000 Cost: 0.440472
Epoch  600/1000 Cost: 0.403240
Epoch  700/1000 Cost: 0.367535
Epoch  800/1000 Cost: 0.332145
Epoch  900/1000 Cost: 0.296141
Epoch 1000/1000 Cost: 0.260252


In [19]:
# Class Implement

class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)  # Output이 3

    def forward(self, x):
        return self.linear(x)

In [20]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [21]:
nb_epochs = 1000

for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.197643
Epoch  100/1000 Cost: 0.638265
Epoch  200/1000 Cost: 0.557551
Epoch  300/1000 Cost: 0.503918
Epoch  400/1000 Cost: 0.459924
Epoch  500/1000 Cost: 0.420655
Epoch  600/1000 Cost: 0.383823
Epoch  700/1000 Cost: 0.347939
Epoch  800/1000 Cost: 0.311875
Epoch  900/1000 Cost: 0.275316
Epoch 1000/1000 Cost: 0.245338
